In [15]:
import pandas as pd
import numpy as np
import matplotlib as mpl
import seaborn as sns
import folium as fl

In [54]:
df=pd.read_excel("Standorte_tracking.xlsx")
df.head(10)

,Name,Land,Stadt,Products in Use,Carbon_footprint
0,Maggi,Switzerland,Zürich,"Heat Pump, Chiller",4t
1,Dr.Oetker,Deutschland,Ettlingen,Compressor,5t
2,Roche,Deutschland,Mannheim,"Heat Pump, Chiller",16t
3,Fächerbad,Deutschland,Pforzheim,Compressor,2t
4,Hoepfner,Deutschland,Karlsruhe,"Heat Pump, Chiller, and additional compressor",4t
5,Moninger,Deutschland,Karlsruhe,Compressor,4t
6,Schwarzwald_Milch,Deutschland,Freiburg,"Heat Pump, Chiller",5t
7,Innocent,Deutschland,München,"Heat Pump, Chiller, and additional compressor",5t
8,Bayer,Deutschland,Grenzach,"Heat Pump, Chiller, and additional compressor",16t
9,BASF,Deutschland,Ludwigshafen,"Heat Pump, Chiller, and additional compressor",16t


In [55]:
map=np.array(df)

In [56]:
loc=map[:,1:3]
print(loc)
dfloc=pd.DataFrame(loc,columns=['Country','City'])
dfloc.dropna(inplace=True)
dfloc = dfloc.apply(','.join, axis=1)
print(dfloc)
dfloc=pd.DataFrame(dfloc,columns=['Loc'])

[['Switzerland' 'Zürich']
 ['Deutschland' 'Ettlingen']
 ['Deutschland' 'Mannheim']
 ['Deutschland' 'Pforzheim']
 ['Deutschland' 'Karlsruhe']
 ['Deutschland' 'Karlsruhe']
 ['Deutschland' 'Freiburg']
 ['Deutschland' 'München']
 ['Deutschland' 'Grenzach']
 ['Deutschland' 'Ludwigshafen']
 ['Deutschland' 'Hamburg']
 ['Deutschland' 'Berlin']
 ['Deutschland' 'Köln']]
0           Switzerland,Zürich
1        Deutschland,Ettlingen
2         Deutschland,Mannheim
3        Deutschland,Pforzheim
4        Deutschland,Karlsruhe
5        Deutschland,Karlsruhe
6         Deutschland,Freiburg
7          Deutschland,München
8         Deutschland,Grenzach
9     Deutschland,Ludwigshafen
10         Deutschland,Hamburg
11          Deutschland,Berlin
12            Deutschland,Köln
dtype: object


In [57]:
from geopy.geocoders import Nominatim
geolocator = Nominatim(user_agent='myGeocoder')
location=geolocator.geocode('Paris,France')
print('Latitude = {}, Longitude = {}'.format(location.latitude, location.longitude))

Latitude = 48.8588897, Longitude = 2.3200410217200766


In [58]:
from geopy.extra.rate_limiter import RateLimiter

# 1 - conveneint function to delay between geocoding calls
geocode = RateLimiter(geolocator.geocode, min_delay_seconds=1)
# 2- - create location column
dfloc['location'] = dfloc['Loc'].apply(geocode)
# 3 - create longitude, laatitude and altitude from location column (returns tuple)
dfloc['point'] = dfloc['location'].apply(lambda loc: tuple(loc.point) if loc else None)
# 4 - split point column into latitude, longitude and altitude columns
dfloc[['latitude', 'longitude', 'altitude']] = pd.DataFrame(dfloc['point'].tolist(), index=dfloc.index)

In [62]:
dfloc['Name']=df['Name']
dfloc['Products in Use']=df['Products in Use']
dfloc['Carbon footprint']=df['Carbon_footprint']

In [68]:
import folium
from folium.plugins import MarkerCluster
from folium.plugins import FastMarkerCluster
map1 = folium.Map(
    location=[20,0],
    tiles='OpenStreetMap',
    zoom_start=6,
)
#FastMarkerCluster(data=list(zip(dfloc['latitude'].values, dfloc['longitude'].values,))).add_to(map1)
folium.LayerControl().add_to(map1)
sw = dfloc[['latitude', 'longitude']].min().values.tolist()
ne = dfloc[['latitude', 'longitude']].max().values.tolist()

map1.fit_bounds([sw, ne]) 

dfloc.apply(lambda row:folium.Marker(location=[row['latitude'], row['longitude']],tooltip='Name: {}<br>Products: {}<br> Crabon Footprint: {}'.format(row['Name'],row["Products in Use"],row["Carbon footprint"])).add_to(map1), axis=1)

map1

In [69]:
map1.save('Map_tracking.html')